In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
pd.set_option("display.max_columns", 85)
pd.set_option("display.max_rows", 85)

### Talkapalooza - TL BL SS 23

# Data preparation:

Based on previous notebook, reorder audio files in a way that they can be used for training a Deep Learning model.

Read the csv

In [22]:
words_df = pd.read_csv('../output/wordbank_crossref.csv')

In [33]:
word_list = words_df['word']

pattern_asterisk = re.compile('\*')
pattern_indicator = re.compile('\s\(.*\)')

word_list = [re.sub(pattern_asterisk, '', str(x)) for x in word_list]
word_list = [re.sub(pattern_indicator, '', str(x)) for x in word_list]

In [66]:
import os
import shutil
from pathlib import Path

root = "../.source/lingualibre_en/"
target = "../.source/labelled_audio/"

In [87]:
pattern_path = re.compile(".*/lingualibre_en/")
pattern_file = re.compile("\.ogg")

In [88]:
word_counter = 0
author_counter = 0

for path, subdirs, files in os.walk(root):
    for file in files:
        str_author = re.sub(pattern_path, '', str(path))
        str_author = re.sub(r'(?<=\w)\.', '_', str_author)
        str_author = re.sub(r'/s', '-', str_author)
        str_word = re.sub(pattern_file, '', str(file))

        if str_word not in word_list:
            continue

        src = os.path.abspath(Path(path, file))
        dst = os.path.abspath(Path(target) / str_word)
        dst = dst + '/'

        try:
            shutil.copyfile(src = src, dst = f"{dst}/{str_author}.ogg")
            author_counter += 1

        except:
            os.makedirs(os.path.dirname(dst), exist_ok=True)
            word_counter += 1
            shutil.copy(src = src, dst = f"{dst}/{str_author}.ogg")
            author_counter += 1
else:
    print(f'Total of {author_counter} recordings across {word_counter} labels.')


Total of 1205 recordings across 471 labels.


## Verifying results:

In [90]:
pattern_target = re.compile(".*/labelled_audio/")

In [95]:
labels_df = pd.DataFrame()

In [96]:
for path, subdirs, files in os.walk(target):
    for file in files:
        str_label = re.sub(pattern_target, '', str(path))
        str_author = re.sub(pattern_file, '', str(file))
        temp_df = pd.DataFrame([[str_label, str_author]], columns=['label', 'author'])
        labels_df= pd.concat([labels_df, temp_df])

In [100]:
labels_df = labels_df.reset_index().drop(columns='index')

In [101]:
labels_df.head(10)

,label,author
0,vacuum,Back ache
1,above,Nattes à chat
2,above,Back ache
3,where,She animates
4,where,Lirazelf
5,where,Greenman
6,where,SpringProof
7,where,Nattes à chat
8,where,Back ache
9,big,Opsylac
